# Run MAST to get Exon level p-value

In [ ]:
library(dplyr)
library(Seurat)
library(patchwork)

In [ ]:
seurat_obj <- readRDS(file = "Feature_PAAD_N3_T10_T8_T20_seurat_object_main.rds")

In [ ]:
Idents(seurat_obj) <- "Condition"

In [ ]:
seurat_obj <- NormalizeData(seurat_obj, normalization.method = "LogNormalize", scale.factor = 10000)

In [ ]:
for (cluster_name in c('2')) {
    sub_seurat <- subset(seurat_obj, subset=leiden_0_1 == cluster_name)
    DE_MAST <- FindMarkers(sub_seurat, ident.1="Cancer", ident.2="Normal", test.use="MAST", logfc.threshold=0.5)
    write.csv(DE_MAST, file = paste0("./MAST/", "Feature_MAST_","leiden_0_1", "_cluster_", cluster_name,".csv"), row.names = TRUE)
}